<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [3]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [5]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


In [6]:
import requests

# Define the URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/API_call_spacex_api.json"

# Send a GET request to the URL
response = requests.get(url)

# Display the response object
print(response)

# Optionally, you can check the status code of the response
print("Status Code:", response.status_code)

# If the request was successful (status code 200), you can also print the content of the response
if response.status_code == 200:
    print("Response Content:", response.content)

<Response [200]>
Status Code: 200
Response Content: b'[{"fairings": {"reused": false, "recovery_attempt": false, "recovered": false, "ships": []}, "links": {"patch": {"small": "https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png", "large": "https://images2.imgbox.com/40/e3/GypSkayF_o.png"}, "reddit": {"campaign": null, "launch": null, "media": null, "recovery": null}, "flickr": {"small": [], "original": []}, "presskit": null, "webcast": "https://www.youtube.com/watch?v=0a_00nJ_Y88", "youtube_id": "0a_00nJ_Y88", "article": "https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html", "wikipedia": "https://en.wikipedia.org/wiki/DemoSat"}, "static_fire_date_utc": "2006-03-17T00:00:00.000Z", "static_fire_date_unix": 1142553600, "tbd": false, "net": false, "window": 0, "rocket": "5e9d0d95eda69955f709d1eb", "success": false, "details": "Engine failure at 33 seconds and loss of vehicle", "crew": [], "ships": [], "capsules": [], "payloads": ["5eb0e4b5b6c3bb0006eeb1e1"], "launchpa

### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [8]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# Define the URL for the Falcon 9 Launch Wikipedia page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'

# Perform the GET request to the URL
response = requests.get(static_url)

# Display the response object and status code
print(response)
print("Status Code:", response.status_code)


<Response [200]>
Status Code: 200


Create a `BeautifulSoup` object from the HTML `response`


In [9]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup
import requests

# Define the URL for the Falcon 9 Launch Wikipedia page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'

# Perform the GET request to the URL
response = requests.get(static_url)

# Create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Display the BeautifulSoup object to verify
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Falcon 9 - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpre

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [10]:
# Use soup.title attribute
# Print the page title to verify if the BeautifulSoup object was created properly
print(soup.title.string)


Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [11]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
from bs4 import BeautifulSoup
import requests

# Define the URL for the Falcon 9 Launch Wikipedia page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'

# Perform the GET request to the URL
response = requests.get(static_url)

# Create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Use the find_all function to find all table elements
html_tables = soup.find_all('table')

# Display the number of tables found
print(f"Number of tables found: {len(html_tables)}")

# Optionally, display the first table to inspect its structure
print(html_tables[0].prettify())


Number of tables found: 25
<table class="infobox hproduct">
 <caption class="infobox-title">
  Falcon 9
 </caption>
 <tbody>
  <tr>
   <td class="infobox-image" colspan="2">
    <span class="mw-default-size" typeof="mw:File/Frameless">
     <a class="mw-file-description" href="/wiki/File:Falcon_9_logo.svg">
      <img alt="Logo of the Falcon 9" class="mw-file-element" data-file-height="162" data-file-width="230" decoding="async" height="49" src="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/70px-Falcon_9_logo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/105px-Falcon_9_logo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/a/a9/Falcon_9_logo.svg/140px-Falcon_9_logo.svg.png 2x" width="70"/>
     </a>
    </span>
   </td>
  </tr>
  <tr>
   <td class="infobox-image" colspan="2">
    <span class="mw-default-size" typeof="mw:File/Frameless">
     <a class="mw-file-description" href="/wiki/File:SpaceX_Demo-2_Launch_(

Starting from the third table is our target table contains the actual launch records.


In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

from bs4 import BeautifulSoup
import requests

# Define the URL for the Falcon 9 Launch Wikipedia page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'

# Perform the GET request to the URL
response = requests.get(static_url)

# Create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all table elements
html_tables = soup.find_all('table')

# Function to extract column name from header element
def extract_column_from_header(th):
    return th.text.strip()

# Initialize the list for column names
column_names = []

# Assuming the first table is the one we are interested in
first_launch_table = html_tables[0]

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Display the extracted column names
print(column_names)


['Function', 'Manufacturer', 'Country of origin', 'Cost per launch', 'Size', 'Height', 'Diameter', 'Mass', 'Stages', 'Capacity', 'Payload to LEO', 'Orbital inclination', 'Mass', 'Payload to GTO', 'Orbital inclination', 'Mass', 'Payload to Mars', 'Mass', 'Associated rockets', 'Based on', 'Derivative work', 'Launch history', 'Status', 'Launch sites', 'Total launches', 'Success(es)', 'Failure(s)', 'Partial failure(s)', 'Notable outcome(s)', 'Landings', 'First flight', 'Last flight', 'First stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant', 'Second stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant']


Check the extracted column names


In [14]:
print(column_names)

['Function', 'Manufacturer', 'Country of origin', 'Cost per launch', 'Size', 'Height', 'Diameter', 'Mass', 'Stages', 'Capacity', 'Payload to LEO', 'Orbital inclination', 'Mass', 'Payload to GTO', 'Orbital inclination', 'Mass', 'Payload to Mars', 'Mass', 'Associated rockets', 'Based on', 'Derivative work', 'Launch history', 'Status', 'Launch sites', 'Total launches', 'Success(es)', 'Failure(s)', 'Partial failure(s)', 'Notable outcome(s)', 'Landings', 'First flight', 'Last flight', 'First stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant', 'Second stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [15]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

KeyError: 'Date and time ( )'

In [25]:
from bs4 import BeautifulSoup
import requests

# Define the URL for the Falcon 9 Launch Wikipedia page
static_url = 'https://en.wikipedia.org/wiki/Falcon_9'

# Perform the GET request to the URL
response = requests.get(static_url)

# Create a BeautifulSoup object from the response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all table elements
html_tables = soup.find_all('table')

# Function to extract column name from header element
def extract_column_from_header(th):
    return th.text.strip()

# Initialize the list for column names
column_names = []

# Assuming the first table is the one we are interested in
first_launch_table = html_tables[0]

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for th in first_launch_table.find_all('th'):
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
        column_names.append(name)

# Display the extracted column names
print("Column Names: ", column_names)

# Create the dictionary from column names
launch_dict = dict.fromkeys(column_names)

# Remove an irrelevant column if it exists
if 'Date and time ( )' in launch_dict:
    del launch_dict['Date and time ( )']

# Initialize the launch_dict with each value to be an empty list
for key in launch_dict.keys():
    launch_dict[key] = []

# Add the new columns
new_columns = ['Version Booster', 'Booster landing', 'Date', 'Time']
for col in new_columns:
    launch_dict[col] = []

# Display the initialized launch_dict keys to verify
print("Initialized launch_dict Keys: ", launch_dict.keys())


Column Names:  ['Function', 'Manufacturer', 'Country of origin', 'Cost per launch', 'Size', 'Height', 'Diameter', 'Mass', 'Stages', 'Capacity', 'Payload to LEO', 'Orbital inclination', 'Mass', 'Payload to GTO', 'Orbital inclination', 'Mass', 'Payload to Mars', 'Mass', 'Associated rockets', 'Based on', 'Derivative work', 'Launch history', 'Status', 'Launch sites', 'Total launches', 'Success(es)', 'Failure(s)', 'Partial failure(s)', 'Notable outcome(s)', 'Landings', 'First flight', 'Last flight', 'First stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant', 'Second stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant']
Initialized launch_dict Keys:  dict_keys(['Function', 'Manufacturer', 'Country of origin', 'Cost per launch', 'Size', 'Height', 'Diameter', 'Mass', 'Stages', 'Capacity', 'Payload to LEO', 'Orbital inclination', 'Payload to GTO', 'Payload to Mars', 'Associated rockets

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [17]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

In [21]:
extracted_row = 0

# Extract each table 
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Get table row 
    for rows in table.find_all("tr"):
        # Check to see if first table heading is a number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False
        
        # Get table element 
        row = rows.find_all('td')
        
        # If it is a number, save cells in a dictionary 
        if flag:
            extracted_row += 1
            
            # Flight Number value
            flight_number = rows.th.text.strip()
            launch_dict['Flight No.'].append(flight_number)
            
            # Date value
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
              
            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [22]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [23]:
import pandas as pd

# Create a DataFrame from the launch_dict
df = pd.DataFrame({ key: pd.Series(value) for key, value in launch_dict.items() })

# Display the first few rows of the DataFrame
print(df.head())


Empty DataFrame
Columns: [Function, Manufacturer, Country of origin, Cost per launch, Size, Height, Diameter, Mass, Stages, Capacity, Payload to LEO, Orbital inclination, Payload to GTO, Payload to Mars, Associated rockets, Based on, Derivative work, Launch history, Status, Launch sites, Total launches, Success(es), Failure(s), Partial failure(s), Notable outcome(s), Landings, First flight, Last flight, First stage, Powered by, Maximum thrust, Specific impulse, Burn time, Propellant, Second stage, Version Booster, Booster landing, Date, Time]
Index: []

[0 rows x 39 columns]


/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
